In [5]:
from ipynb.fs.full.Infer_propagation_path_functions import *

from pymongo import MongoClient

In [6]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [7]:
COLLECTION_NAME = 'data_false_8'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[264361128,
 965856403129745408,
 2371260104,
 1378023550687268872,
 20478016,
 1274557407130083328,
 15987245,
 1021401793266372608,
 1320845430,
 1224685021866483714,
 959953351382364161,
 1189099063054143489,
 1519014456,
 284677980,
 1240653191655698432,
 784866246772727808,
 2263663771,
 1260952232050036736,
 1147021327,
 93930925,
 337286471,
 4831347854,
 1327793345393881089,
 827334883185799168,
 1348293669946851329,
 34922026,
 585777737,
 874807324510113792,
 47598596,
 1427077376,
 3872346202,
 172990190,
 1321831977083932678,
 2670696570,
 26946959,
 20878297,
 396161595,
 1283099384163586049,
 2800470003,
 952050248578486272,
 760149872322510848,
 1275520387460157440,
 2745094971,
 1272473084168192001,
 4926435839,
 858118388,
 882764425693933568,
 41924468,
 717674874,
 252365681,
 3556262056,
 79960355,
 1215397509721313280,
 244164131,
 1350566381654306816,
 23633315,
 20208407,
 3222204881,
 810723502768713728,
 1273638001269248001,
 821922018937290752,
 90877554766843

In [8]:
len(db['followers'].find_one({'user_id': 264361128})['followers'])

462880

In [ ]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)
    print(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{264361128}
264361128 Tue Apr 27 17:56:36 +0000 2021
264361128
264361128
[971176632621953024, 857029745464180737, 1375381514, 130713612, 1105774474555785217, 2218639374, 1147021327, 307562513, 35004434, 978692921527586818, 42616853, 42242071, 1252909080, 289703965, 437895198, 1055188190238257152, 1336812699473354754, 774337735897198592, 1314347481963147266, 53231656, 1016749275886206976, 1364929426413277184, 153968684, 1283099384163586049, 166514736, 66070578, 1075194934695731200, 2391949368, 9508922, 1094449606392344576, 912097800661426176, 780530249406697472, 705561147156508673, 1461493826, 20478016, 33110084, 1029780752051920898, 798351157001388033, 1241172871512231936, 1382059887358844930, 950862222661771264, 962156417300938752, 1379909255009042439, 2871410776, 318697561, 3872346202, 2214273115, 30640220, 1012931887227179008, 1228937546970750976, 4758155359, 1379803730128822275, 1356348960852627456, 2278137954, 23533665, 799406180, 948232248867934208, 1247866757643190273, 132941094

In [5]:
users= [264361128]

In [6]:
for user in users:
    print(user, get_followers_len(user, db))

TypeError: 'NoneType' object is not subscriptable

In [7]:
for user in users:
    print(user, get_cascade_size(user, COLLECTION_NAME, db))

264361128 3895


In [8]:
# Check if the text is same
db[COLLECTION_NAME].distinct('retweet_count')

[0, 4570]

In [9]:
db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')

[4570]

In [13]:
db['data_false_7'].distinct('retweeted_status.quoted_status.retweet_count')

[2753]

In [10]:
# Check if the text is same
cache = db[COLLECTION_NAME].distinct('retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweet_count':x})['text'])
    print('*'*5)

[0, 4570]
RT @Liz_Wheeler: Joe Rogan is correct about the COVID vaccine for kids &amp; young healthy people.

https://t.co/AxGwFxAMVB
*****
Joe Rogan is correct about the COVID vaccine for kids &amp; young healthy people.

https://t.co/AxGwFxAMVB
*****


In [11]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.retweet_count':x})['id'])
    print('*'*5)

[4570]
1387103453000830977
*****


In [12]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.quoted_status.retweet_count':x})['retweeted_status.quoted_status.id'])
    print('*'*5)

[]


In [13]:
db['timeline_extended'].count()

C:\Users\Akila\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


616